In [1]:
from glob import glob
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import xgcm
from xorca.lib import load_xorca_dataset
import pickle
import eddytools as et
from cmocean import cm
from scipy.signal import convolve
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
# Obviously it is not a great idea to ignore warnings, however there are quite many
# RuntimeWarnings because of division by 0 in some parts of this notebook. To keep
# the instructive nature of this example notebook, these warnings are ignored.

In [2]:
experiment_name = 'INALT20r.L120-KRS006'#'INALT60.L120-KRS0020'
data_resolution = '1d'

OW_thr_factor =-0.3

In [3]:
tracking_start, tracking_end = "2012-01-01", "2012-12-31"
#periods = [
#    ("2012-01-01", "2012-01-25"), ("2012-01-26", "2012-02-19"), ("2012-02-20", "2012-03-15"),
#    ("2012-03-16", "2012-04-09"), ("2012-04-10", "2012-05-04"), ("2012-05-05", "2012-05-29"),
#    ("2012-05-30", "2012-06-28"), ("2012-10-27", "2012-11-20"), ("2012-11-21", "2012-12-15"),
#    
#    ("2012-06-29", "2012-07-28"), ("2012-07-29", "2012-08-27"), ("2012-08-28", "2012-09-26"), ("2012-09-27", "2012-10-26"),
#]

In [4]:
if experiment_name.startswith("INALT60"):
    Npix_min, Npix_max = 720, 18000
    sigma = 15
    wx = 600
elif experiment_name.startswith("INALT20"):
    Npix_min, Npix_max = 80, 2000
    sigma = 5
    wx = 200

In [5]:
datapath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-0/'   # depth = surface !!!

In [6]:
# Specify parameters for eddy tracking
tracking_parameters = {'model': 'ORCA',
                       'grid': 'latlon',
                       'start_time': tracking_start, # time range start
                       'end_time': tracking_end, # time range end
                       'calendar': 'standard', # calendar, must be either 360_day or standard
                       'dt': 1, # temporal resolution of the data in days
                       'lon1': 0, # minimum longitude of detection region
                       'lon2': 40, # maximum longitude
                       'lat1': -45, # minimum latitude
                       'lat2': -25, # maximum latitude
                       'search_dist': 100., # maximum distance of search ellipse/circle from eddy center in km
                                          # if ellipse: towards the east (if set to 0, it
                                          # will be calculated as (150. / (7. / dt)))
                       # max vel 3.5 m/s --> 50km in 4 hours
                       'search_circle': True, # if True, search in a circle. otherwise use ellipse
                       'eddy_scale_min': 0.5, # minimum factor by which eddy amplitude and area are allowed to change in one timestep
                       'eddy_scale_max': 1.5, # maximum factor by which eddy amplitude and area are allowed to change in one timestep
                       'dict': 0, # eddies dictionary containing detected eddies to be used when not stored in files (set to 0 otherwise)
                       'data_path': datapath, # path to the detected eddies pickle files
                       'file_root': 'Eddies',
                       'file_spec': f'OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}_rolling-{wx}',
                       'ross_path': '/gxfs_work/geomar/smomw523/eddytools/'} # path to rossrad.dat containing Chelton et a1. 1998 Rossby radii

# detected eddies are loaded from file with the filename
# 'trac_param['data_path'] + trac_param['file_root'] + '_'
# + str(datestring) + '_' + trac_param['file_spec'] + '.pickle'
# `datestring` is created from the `time` value of the eddy

In [7]:
if experiment_name.startswith("INALT60"):
    prefix = "2_"
    tracking_parameters['lon1'], tracking_parameters['lon2'] = 0, 40
    tracking_parameters['lat1'], tracking_parameters['lat2'] = -45, -25
elif experiment_name.startswith("INALT20"):
    prefix = "1_"
    tracking_parameters['lon1'], tracking_parameters['lon2'] = -19, 40
    tracking_parameters['lat1'], tracking_parameters['lat2'] = -49, -7

In [8]:
#depth= 0  #corresponding to... 
#depth_index = 0 

mesh_mask = xr.open_dataset(f'/gxfs_work/geomar/smomw523/smoothed_data/{experiment_name}/{prefix}{experiment_name}_mesh_mask.nc') 
indices = np.concatenate((range(0, 11, 10),range(18, 25, 6),range(29, 34, 4),range(36, 40, 3),range(41, 120, 2)))
depth_information = [(round(mesh_mask.nav_lev.values[i]), i) for i in indices]
print(len(depth_information),depth_information)

48 [(0, 0), (21, 10), (54, 18), (91, 24), (132, 29), (173, 33), (210, 36), (253, 39), (286, 41), (323, 43), (365, 45), (412, 47), (465, 49), (525, 51), (592, 53), (668, 55), (753, 57), (848, 59), (952, 61), (1066, 63), (1189, 65), (1321, 67), (1461, 69), (1608, 71), (1762, 73), (1922, 75), (2086, 77), (2255, 79), (2428, 81), (2603, 83), (2782, 85), (2963, 87), (3146, 89), (3331, 91), (3518, 93), (3706, 95), (3896, 97), (4086, 99), (4279, 101), (4472, 103), (4666, 105), (4861, 107), (5057, 109), (5254, 111), (5452, 113), (5651, 115), (5850, 117), (6050, 119)]


# TRACKING

Things to note for the setting of `tracking_parameters`:  
1. `'start_time'` is required to be no earlier than the earliest actual date of the detected eddies. In our case here, for the year 0002 and a 5-day temporal resolution of the data, this is `'0002-01-05'` (The `MITgcm` stores the 5-daily averages at the end of the 5-day period).
2. `'lon1'` and `'lon2'` need to be identical to `'lon1'` and `'lon2'` in `detection_parameters`.  
3. If you stored the detected eddies in files and want to track these, set `'dict': 0`, make sure `'data_path'`, `'file_root'` and `'file_spec'` are set accordingly and use `in_file=True` as an argument to `et.tracking.track()`.. The method will look for files `datapath + file_root + 'YYYYMMDD' + file_spec + '.pickle'`, the date `'YYYYMMDD'` is automatically calculated from `'start_time'`, `'dt'`, and `'end_time'`. You have to make sure that the stored, detected eddies contain that date in their filename (e.g. as defined in the cell above)!  

Some notes on the search distance `search_dist` that is used to determine in what radius to look for a similar eddy at the next time step.  
    - If `search_circle: True`, the algorithm simply searches for similar eddies within a radius of `search_dist` kilometers around the center of the current eddy. This is the simplest method.
    - If `search_circle: False`, the algorithm will determine where to look for similar eddies based on an ellipse with a minor axis of `search_dist` kilometers. If `search_dist: 0`, the minor and major axes of the ellipse are calculated based on the propagation of Rossby waves to account for the fact that eddies will move towards the West (see [Chelton et al., 2011](https://www.sciencedirect.com/science/article/pii/S0079661111000036) for details). In regions with strong currents this might lead to a loss of a lot of tracks, but no sensitivity studies have been conducted... 

In [9]:
# Now we track the eddies, all information needed has to be added to `tracking_parameters`
for (depth, depth_index) in tqdm(depth_information):
    datapath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-{depth}/'
    tracking_parameters['data_path'] = datapath
    tracks = et.tracking.track(tracking_parameters)
    with open(datapath
            + f'Tracks_{tracking_start.replace("-", "")}_{tracking_end.replace("-", "")}_OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}_rolling-{wx}.pickle', 'wb') as f:
        pickle.dump(tracks, f, pickle.HIGHEST_PROTOCOL)
    f.close()

  0%|          | 0/48 [00:00<?, ?it/s]

tracking at time step  42  of  365
tracking at time step  82  of  365
tracking at time step  123  of  365
tracking at time step  163  of  365
tracking at time step  204  of  365
tracking at time step  244  of  365
tracking at time step  285  of  365
tracking at time step  325  of  365


  2%|▏         | 1/48 [00:59<46:50, 59.79s/it]

tracking at time step  42  of  365
tracking at time step  82  of  365
tracking at time step  123  of  365
tracking at time step  163  of  365
tracking at time step  204  of  365
tracking at time step  244  of  365
tracking at time step  285  of  365
tracking at time step  325  of  365


  4%|▍         | 2/48 [01:59<45:54, 59.88s/it]

...

tracking at time step  42  of  365
tracking at time step  82  of  365
tracking at time step  123  of  365
tracking at time step  163  of  365
tracking at time step  204  of  365
tracking at time step  244  of  365
tracking at time step  285  of  365
tracking at time step  325  of  365


 96%|█████████▌| 46/48 [38:04<00:34, 17.18s/it]

tracking at time step  42  of  365
tracking at time step  82  of  365
tracking at time step  123  of  365
tracking at time step  163  of  365
tracking at time step  204  of  365
tracking at time step  244  of  365
tracking at time step  285  of  365
tracking at time step  325  of  365


100%|██████████| 48/48 [38:14<00:00, 47.81s/it]

no eddies found to track


We now have tracked all eddies that met the criteria specified in `tracking_parameters`. Every entry `i` in `tracks[i]` corresponds to one complete track.

# Rest

### Split up the tracking
If detecting and tracking in larger regions, the tracking can take very long. Because computations are often limited in duration on high-performance computing clusters, an option to split up the tracking into several chunks has been added. Note that the tracking still needs to be done serially, one chunk after the other.

We first track the first half of the month, using the function `split_track()`.

The file `tmp_split1` contains the tracks for the first time span as if they were tracked with `track()`. `tracks_split` additionally contains all unfinished tracks and needs to be passed on to the next round of tracking. `terminated1` contains all tracks that have been already terminated and needs to passed on to the next round as well to make sure they don't get tracked again.

The variable `tracks_from_split` now contains the same tracks as the variable `tracks` from the normal tracking function `track()`. `tracks_split2` and `terminated2` are only needed when another period of tracking is to be added.